In [9]:
from sqlalchemy.ext.asyncio import async_sessionmaker, create_async_engine
import sqlalchemy as sqla
from config import env


engine = create_async_engine(
    sqla.URL.create(
        host=env.DB_URL,
        drivername=env.DB_DRIVER,
        username="postgres",
        password="postgres",
        # port=env.DB_PORT,
        port=5433,
        database=env.DB_NAME,
    ),
    pool_pre_ping=True,
)

ss = async_sessionmaker(
    engine,
    expire_on_commit=False,
)()


In [5]:
from sqlalchemy.ext.asyncio import async_sessionmaker, create_async_engine
import sqlalchemy as sqla
from config import env


engine_c = create_async_engine(
    sqla.URL.create(
        host=env.DB_URL,
        drivername=env.DB_DRIVER,
        username="postgres",
        password="postgres",
        # port=env.DB_PORT,
        port=5433,
        database=env.DB_NAME,
    ),
    pool_pre_ping=True,
)

ss_c = async_sessionmaker(
    engine_c,
    expire_on_commit=False,
)()


In [8]:
from sqlalchemy.orm import make_transient
from db.postgresql.models.blog import ProductDoc
from db.postgresql.models.product import Product, ProductEmbedding


async with ss.begin():
    prods = await ss.scalars(sqla.select(Product))

    ps = []
    pd = []
    pe = []

    for p in prods.all():
        ss.expunge(p)
        make_transient(p)

        p_doc = await ss.get_one(ProductDoc, p.id)
        p_embed = await ss.get_one(ProductEmbedding, p.id)

        ss.expunge(p_doc)
        make_transient(p_doc)

        ss.expunge(p_embed)
        make_transient(p_embed)

        id = f"{p.product_types}_{p.id}"
        p.id = id
        p_doc.id = id
        p_embed.id = id

        ps.append(p)
        pd.append(p_doc)
        pe.append(p_embed)


async with ss_c.begin():
    ss_c.add_all(ps)
    ss_c.add_all(pd)
    ss_c.add_all(pe)
    await ss_c.commit()

In [5]:
from db.postgresql import init_db

await init_db()

In [10]:
from db.postgresql.models.product import (
    Product,
    ProductPriceHistory,
    ProductStockHistory,
)
from tqdm.autonotebook import tqdm
from datetime import datetime
import datetime as dt

/tmp/ipykernel_19622/2328573263.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [11]:
MAX_PRICE_CHANGE_AMOUNT = 30
MIN_PRICE_CHANGE_AMOUNT = 5

MAX_STOCK_CHANGE_AMOUNT = 30
MIN_STOCK_CHANGE_AMOUNT = 5

MAX_ITEM_PER_ORDER = 5
MIN_ITEM_PER_ORDER = 3

MAX_AMOUNT_OF_ITEM_IN_ORDER = 5
MIN_AMOUNT_OF_ITEM_IN_ORDER = 3

MAX_PRICE = 40
MIN_PRICE = 5

MAX_SALE = 30
MIN_SALE = 0

MAX_PRICE_CHANGE_INTERVAL_DAY = 70
MIN_PRICE_CHANGE_INTERVAL_DAY = 20


MAX_RESTOCK_INTERVAL = 70
MIN_RESTOCK_INTERVAL = 20

In [12]:
USER_AMOUNT = 100

STAFF_AMOUNT = 40
SHIPPER_AMOUNT = 100

MAX_ORDER_PER_USER = 10
MIN_ORDER_PER_USER = 2

MAX_ORDER_INTERVAL_DAY = 14
MIN_ORDER_INTERVAL_DAY = 2

In [13]:
import random
from passlib.context import CryptContext
import faker

from db.postgresql.models.user_account import UserAccount, UserAccountStatus

fake = faker.Faker()

In [14]:
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")


def generate_phone() -> str:
    num_arr = ["0"]

    for _ in range(9):
        num_arr.append(str(random.randint(0, 9)))

    return "".join(num_arr)


user_password = pwd_context.hash("password")

(trapped) error reading bcrypt version
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/passlib/handlers/bcrypt.py", line 620, in _load_backend_mixin
    version = _bcrypt.__about__.__version__
AttributeError: module 'bcrypt' has no attribute '__about__'


In [15]:
import uuid

pbar = tqdm(desc="Adding customer account", total=USER_AMOUNT)

async with ss.begin():
    for _ in range(USER_AMOUNT):
        username = fake.user_name()
        id = str(uuid.uuid4())

        staff = UserAccount(
            id=id,
            email=fake.email(),
            username=username,
            profile_name=username,
            password=user_password,
            token=username,
            status=random.choices(
                [UserAccountStatus.NORMAL, UserAccountStatus.BANNED],
                [0.9, 0.1],
            )[0],
            address=fake.address(),
            profile_pic_uri="defaultProfile",
            profile_description=" ",
            phone=generate_phone(),
        )

        pbar.update()

        ss.add(staff)

    await ss.flush()


Adding customer account:   0%|          | 0/100 [00:00<?, ?it/s]

In [16]:
import uuid

from db.postgresql.models.staff_account import (
    AccountStatus,
    AccountType,
    EmployeeInfo,
    StaffAccount,
)


async def generate_staff(type: AccountType, amount: int):
    pbar = tqdm(desc="Adding customer account", total=amount - 1)

    id_list = []
    async with ss.begin():
        for _ in range(amount):
            username = fake.user_name()
            id = str(uuid.uuid4())

            id_list.append(id)
            staff = StaffAccount(
                id=id,
                username=username,
                password=user_password,
                type=type,
                token=username,
                status=random.choices(
                    [
                        AccountStatus.ACTIVE,
                        AccountStatus.DISABLE,
                    ],
                    [0.9, 0.1],
                )[0],
            )

            info = EmployeeInfo(
                account_id=id,
                ssn=generate_phone(),
                phonenumber=generate_phone(),
                realname=fake.name(),
                email=fake.email(),
                dob=fake.date_of_birth(),
            )

            pbar.update()

            ss.add(staff)
            ss.add(info)

        await ss.flush()
    return id_list

In [17]:
await generate_staff(AccountType.STAFF, STAFF_AMOUNT)


Adding customer account:   0%|          | 0/39 [00:00<?, ?it/s]

['ee8c58cc-05b2-4f35-ae01-0b9c7c984d3c',
 '5bd2500d-6ff8-42f0-9299-e0a5088f4bcd',
 '8a223f34-aa8b-4c41-91db-c6c629dbe727',
 '08dbf337-541e-4c14-b9af-8c42e4c690ec',
 'b003881a-aed0-4d1e-802d-b880b38465bc',
 'aea5c4e7-d6ae-4e7f-b49a-3a001f227fcc',
 '5472b0a5-e64c-4d08-89de-2ac33a87a081',
 'f7bd3af8-1464-4b9a-bcfa-70321e2f1213',
 'b84eff19-b3a5-4325-ae69-228770c7d6b4',
 'bd652e1d-b5ad-4d09-948f-c9b9228ec717',
 '787d2e43-b182-4cc5-acf0-cb777fe46acc',
 'e267bd48-8ca0-451e-abdf-4ce5b5385b97',
 '33aa9524-5ea8-42e3-853f-3d40dbf0f7f6',
 '3f0b687c-9e25-43df-84a4-832be30935e7',
 'ef1812fd-e9d5-4c77-accc-6a08a0314931',
 '6703df34-8e27-41ca-83cc-76baf5af0ede',
 '7498a5c5-f20d-40e5-a7e1-1abfb93db1ed',
 '8283cd2d-7766-45db-9065-c51c6a22d7fc',
 '63945140-6e25-403c-9721-5d508aef6d8f',
 '6006347e-41a4-4497-a77c-00fde2b84724',
 '2bcf1d9e-08b6-40df-86b6-bc144b1650d0',
 '30e05c3e-407d-45b3-9297-9a50dfa4d110',
 '245603d0-89ce-47e4-9482-afe8688618f8',
 'a1532aaa-e704-460e-861b-b880d38298d1',
 'cc320f6b-b155-

In [18]:
await generate_staff(AccountType.SHIPPER, STAFF_AMOUNT)

Adding customer account:   0%|          | 0/39 [00:00<?, ?it/s]

['5c6e9748-b638-4ef6-8427-3c38ce559877',
 'e4ad2ef8-a65c-45bf-8770-3650dd0e3c62',
 '14fd549f-7a24-4d2a-903f-70b82c419a9a',
 '89da7ee7-b7cb-49c4-8c38-1e1b04c74d64',
 '454a9092-2def-41fe-ac75-e8409b0d52d3',
 '34298cc1-6646-4447-a437-ebe4176cb70a',
 '4891a57f-8fbf-48d5-838b-d733b92a98d7',
 'a17355db-3162-4e62-9544-3b7c375e8690',
 'b5b37fc7-086c-4675-b288-f50cb61bd170',
 'c700f10d-a498-4e88-bd6f-d2b4c1599949',
 'dc8d65e2-4a22-4c1e-b781-415b6a2d77c1',
 '9b897d5f-d1b5-4da3-a765-d23357e08379',
 '7ed3821a-6a17-4772-91ea-40fffa16227d',
 '443f3e57-3e47-4547-b70e-232e8d4da3f7',
 '8023248d-10e3-4baf-9b01-1104467e9db8',
 'f79205a4-8887-48bf-b974-c543d80054c5',
 '8f1da888-2cb6-4f35-826e-7c50bf7bcceb',
 '7c8b7ca4-ee89-4cd1-a7ad-682b896d0f5f',
 'd945406b-7eaf-4947-99f2-d3fbaa12d6c5',
 '6940b5f6-5dc0-4cab-893d-42757bc50158',
 '9979cf8f-610e-4d24-88fa-9867c2482b93',
 'bb17ce8a-dfaa-40d8-a989-ed1a087c29e1',
 '2b3561ec-9958-4ab6-bc54-9d0b13615dcd',
 'c16fd80a-12db-4fb6-a017-f8a3fc13be41',
 'f14e1693-21d2-

In [19]:
from db.postgresql.models.staff_account import StaffAccount, AccountType

async with ss.begin():
    product_amount = await ss.scalar(sqla.select(sqla.func.count(Product.id))) or 0
    product_ids = (await ss.scalars(sqla.select(Product.id))).all()
    account_ids = (await ss.scalars(sqla.select(UserAccount.id))).all()
    account_amount = (
        await ss.scalar(
            sqla.select(
                sqla.func.count(
                    UserAccount.id,
                )
            )
        )
        or 0
    )
    shippers = (
        await ss.scalars(
            sqla.select(StaffAccount.id).filter(
                StaffAccount.type == AccountType.SHIPPER
            )
        )
    ).all()

    staffs = (
        await ss.scalars(
            sqla.select(StaffAccount.id).filter(
                StaffAccount.type == AccountType.STAFF,
            )
        )
    ).all()

In [20]:
from datetime import time, timedelta

from db.postgresql.models.shipper import ShipperAvailbility


async with ss.begin():
    for s in shippers:
        start_time = random.randint(1, 23)
        end_time = (start_time + random.randint(4, 7)) % 24

        start_ship = time(start_time)

        end_ship = time(end_time)

        sa = ShipperAvailbility(
            id=str(s),
            start_shift=start_ship,
            end_shift=end_ship,
            occupied=fake.boolean(20),
        )
        ss.add(sa)

    await ss.commit()

In [21]:
async with ss.begin():
    pbar_prod = tqdm(total=product_amount - 1)
    pbar_sub = tqdm()

    current_day = datetime.now()

    for prod in product_ids:
        pbar_prod.set_description(f"Product : {prod}")

        price_amount = random.randint(
            MIN_PRICE_CHANGE_AMOUNT,
            MAX_PRICE_CHANGE_AMOUNT,
        )
        stock_amount = random.randint(
            MIN_STOCK_CHANGE_AMOUNT,
            MAX_STOCK_CHANGE_AMOUNT,
        )

        pbar_sub.reset(price_amount)
        pbar_sub.set_description("Generate price")

        prev_date = current_day
        for _ in range(price_amount):
            price = round(random.uniform(MIN_PRICE, MAX_PRICE), 2)
            sale_percent = round(random.uniform(MIN_SALE, MAX_SALE), 2)

            day_offset = dt.timedelta(
                days=random.randint(
                    MIN_PRICE_CHANGE_INTERVAL_DAY, MAX_PRICE_CHANGE_INTERVAL_DAY
                )
            )

            prev_date -= day_offset

            price_history = ProductPriceHistory(
                product_id=prod,
                date=prev_date,
                price=price,
                sale_percent=sale_percent,
            )

            ss.add(price_history)

            pbar_sub.update()

        pbar_sub.reset(stock_amount)
        pbar_sub.set_description("Generate stock")
        prev_date = current_day
        for _ in range(stock_amount):
            price = round(random.uniform(MIN_PRICE, MAX_PRICE) * 0.7, 2)
            stock = random.randint(
                MIN_STOCK_CHANGE_AMOUNT,
                MAX_STOCK_CHANGE_AMOUNT,
            )
            day_offset = dt.timedelta(
                days=random.randint(MIN_RESTOCK_INTERVAL, MAX_RESTOCK_INTERVAL)
            )

            prev_date -= day_offset
            stock = ProductStockHistory(
                product_id=prod,
                in_stock=stock,
                in_price=price,
                date=prev_date,
            )

            ss.add(stock)

            pbar_sub.update()

        pbar_prod.update()

    await ss.flush()

  0%|          | 0/22 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [22]:
import uuid
from db.postgresql.models.order_history import (
    OrderHistory,
    OrderHistoryItems,
    OrderProcess,
    PaymentStatus,
    OrderStatus,
)
from db.postgresql.models.transaction import PaymentTransaction
from services.shipper import ShippingStatus

pbar = tqdm(total=account_amount - 1, desc="Generate per user")
pbar_sub = tqdm(desc="Order:")
async with ss.begin():
    for i in account_ids:
        order_amount = random.randint(MIN_ORDER_PER_USER, MAX_ORDER_PER_USER)

        current_day = datetime.now()

        prev_date = current_day

        pbar_sub.reset(order_amount - 1)
        for _ in range(order_amount):
            item_for_order = random.randint(
                MIN_ITEM_PER_ORDER,
                MAX_ITEM_PER_ORDER,
            )

            day_offset = dt.timedelta(
                days=random.randint(
                    MIN_ORDER_INTERVAL_DAY,
                    MAX_ORDER_INTERVAL_DAY,
                )
            )

            prev_date -= day_offset

            prods = list(
                (
                    await ss.execute(
                        sqla.select(ProductPriceHistory.product_id)
                        .filter(ProductPriceHistory.date < prev_date)
                        .distinct(ProductPriceHistory.product_id)
                    )
                ).scalars()
            )

            if len(prods) < item_for_order:
                break

            order_items = random.sample(prods, item_for_order)

            total_price = 0

            order_id = uuid.uuid4()

            order_items_db = []
            for item in order_items:
                quantity = random.randint(1, 5)
                prod_price = await ss.scalar(
                    sqla.select(ProductPriceHistory)
                    .filter(
                        ProductPriceHistory.product_id == item,
                        ProductPriceHistory.date < prev_date,
                    )
                    .order_by(ProductPriceHistory.date.desc())
                    .limit(1)
                )

                if not prod_price:
                    raise Exception()

                total_price += (prod_price.price * prod_price.sale_percent) * quantity

                order_items_db.append(
                    OrderHistoryItems(
                        order_history_id=order_id,
                        product_id=prod_price.product_id,
                        date=prod_price.date,
                        quantity=quantity,
                    )
                )

            total_price = round(total_price, 2)
            order_status = (
                random.choices(
                    [
                        OrderStatus.CANCELLED,
                        OrderStatus.ON_CONFIRM,
                        OrderStatus.ON_PROCESSING,
                        OrderStatus.ON_SHIPPING,
                        OrderStatus.SHIPPED,
                        OrderStatus.DELIVERED,
                    ],
                    [
                        0.05,
                        0.05,
                        0.05,
                        0.55,
                        0.2,
                        0.1,
                    ],
                ),
            )[0][0]
            order = OrderHistory(
                id=order_id,
                user_id=i,
                order_date=prev_date,
                delivery_address=fake.address(),
                note=fake.sentence(5),
                total_price=total_price,
                receiver=fake.name(),
                phonenumber=generate_phone(),
                coupon=None,
                payment_method="COD",
                payment_status=PaymentStatus.PENDING,
                order_status=order_status,
            )

            ss.add(order)
            ss.add_all(order_items_db)

            if order_status not in (OrderStatus.ON_CONFIRM, OrderStatus.CANCELLED):
                confirm_date = prev_date + dt.timedelta(hours=random.randint(10, 48))

                shipping_date = confirm_date + dt.timedelta(
                    hours=random.randint(10, 24)
                )

                status = None

                match order_status:
                    case OrderStatus.ON_PROCESSING:
                        shipping_date = None
                        status = random.choice([
                            ShippingStatus.ACCEPTED,
                            ShippingStatus.REJECTED,
                        ])
                    case OrderStatus.ON_SHIPPING:
                        status = ShippingStatus.ON_SHIPPING
                    case OrderStatus.SHIPPED | OrderStatus.DELIVERED:
                        status = ShippingStatus.DELIVERED

                process = OrderProcess(
                    order_id=order_id,
                    confirm_date=confirm_date,
                    process_by=random.choice(staffs),
                    shipping_date=shipping_date,
                    deliver_by=random.choice(shippers),
                    status=status,
                )
                ss.add(process)

            if order_status == OrderStatus.DELIVERED:
                payment = PaymentTransaction(
                    order_id=order.id,
                    create_time=order.order_date,
                    amount=order.total_price,
                    status=PaymentStatus.RECEIVED,
                )
                ss.add(payment)

            pbar_sub.update()
        pbar.update()

    await ss.commit()


Generate per user:   0%|          | 0/99 [00:00<?, ?it/s]

Order:: 0it [00:00, ?it/s]